> 1. Introspection
1. Functional programming
    * map-reduced
    * read_int
1. Unit Testing and TDD
    * use cyber-dojo http://cyber-dojo.org/
    * use TDD to implement read_int
1. Iterator and Generator
    * mock command line system
1. Run in parallel
    * batch downloading tool
    * Multi threading
    * Multi processing
    * Gevent
        * BBS like chating service
1. Design patterns
    * Mixin
    * Singleton
    * Decorator
1. Regrex Expression (?)
1. Meta Class (?)


# Python Intermediate

![python](https://upload.wikimedia.org/wikipedia/commons/c/c3/Python-logo-notext.svg)


_talk is cheap, show me the code_

--- Linus Torvalds